# Session 6: Web Scraping 1

## Recap
In session 5 you briefly touched upon extracting data from the internet
- You heard about HTTP requests: The computer's way to communicate with the website and underlying server
- You learned about HTML: The language behind a website
- You worked with APIs: A way to retrieve structured data from websites
    - You learned how to send requests to an API and in return receive the desired data
    - You learned how to deal with the JSON files that is usually how the data is sent

In the next three sessions we will build on your newly acquired skills

You will learn how to extract data from websites when no API is available

It will open up a whole new world of data possibilities!

## Required readings

- [Beautiful Soup: Build a Web Scraper With Python](https://realpython.com/beautiful-soup-web-scraper-python/)

- [A Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)

- Shiab, Nael. 2015. [On the Ethics of Web Scraping and Data Journalism](http://gijn.org/2015/08/12/on-the-ethics-of-web-scraping-and-data-journalism/). Global Investigative Journalism Network.

## Overview of Session 6

Today, we will learn about interacting with websites and extracting their unstructured data (web scraping). In particular, we will cover:
1. Basics of web scraping:
    - What is web scraping?
    - How is a webpage built? How does a computer "see" a webpage?
    - Ethical considerations
2. Mapping the URLs of the webpages you want to scrape:
    - How can we systematically go through all the webpages and scrape their content?
        - You will learn to exploit the pattern in the URLs
3. Connecting to a webpage:
    - How do we extract the HTML-string behind the webpage?
4. Good practices of web scraping
    - Limit the rate of your calls to the webpage
    - Logging
    - Handle exceptions
5. Exploiting the webpage's own data requests to extract data easily
    - Use the network panel in Chrome Developer Tools to locate the data

## What is web scraping?

Web scraping is the practice of extracting information from websites in an automated and structured way

- The internet is the biggest source of information/data you can find! [90% of the data on the internet has been created since 2016](https://blog.microfocus.com/how-much-data-is-created-on-the-internet-each-day/)
- Web scraping unlock this new world of data possibilities 
- The biggest limitation is your own imagination!

#### One note:
- Learning how to web scrape may be frustrating in the beginning
    - It is a new way of thinking data 
    - It requires technological knowledge
- Keep working with it!
    - I promise you; you will master it
    - And the internet will never be the same again

## How a human sees a webpage vs. how a computer sees it

### [www.jobnet.dk](https://job.jobnet.dk/CV/FindWork?Offset=0&SortValue=BestMatch)

How a human sees a webpage             |  How a computer sees a webpage
:-------------------------:|:-------------------------:
![](https://drive.google.com/uc?exportview&id=1cbrC303j-gQnXbXyTEQBPT2xH7kgz6Cy)  |  ![](https://drive.google.com/uc?export=view&id=1VFlfDcJHCzbtmkpr4kvXzGecrDE7KmLY)

## Ethical Considerations (rule of thumbs)
1. Fair use: Take only the stuff you need
2. Be careful with copyrighted material
3. If a regular user can’t access it, we shouldn’t try to get it: [That is considered hacking](https://www.dr.dk/nyheder/penge/gjorde-opmaerksom-paa-cpr-hul-nu-bliver-han-politianmeldt-hacking)
4. If monetizing on the data, be careful not to be in direct competition with whom you are taking the data from
5. LinkedIn case: [Scraping data on LinkedIn is legal](https://gizmodo.com/linkedin-scraping-data-legal-court-case-1848811335)
6. Don't hit it too fast: Essentially a DENIAL OF SERVICE attack (DOS); [Again considered hacking](https://www.dr.dk/nyheder/indland/folketingets-hjemmeside-ramt-af-hacker-angreb)

## The Web Scraping Recipe

Three (main) steps in scraping:
1. **MAPPING (this session)**: Find URLs of the webpages containing the information you want.
2. **DOWNLOADING (this session)**: Download the HTML of the webpages.
3. **PARSING (session 7)**: Extract the information from the HTML. 

### What browser to use?
- Lectures and exercises are solely based on Chrome as my browser --> I recommend you to use Chrome as well. 
- All browsers (Chrome, Firefox, Safari, Edge etc.) can be used to investigate the webpages like you will learn to do here
    - The practice might differ a bit

In [3]:
import requests
import time
import tqdm
import pandas as pd
import os
import json

# Video 6.1: Mapping URLs and downloading webpage content

## 1. Mapping: How do we find the relevant URLs

### Navigating websites to collect links

How can you automate the navigation of links?

### Building URLs using a recognizable pattern
A nice trick is to understand how URLs are constructed to communicate with a server

This will allow us to navigate the page:

* / is like folders on your computer.
* ? entails the start of a query with parameters 
* = defines a variable: e.g. page=1000 or offset = 100 or showNumber=20
* & separates different parameters.
* \+ is html for whitespace

Lets look at how [www.jobindex.dk](https://www.jobindex.dk/jobsoegning) does it:
- We simply click around and take note at how the address line changes

#### We want to create the URLs for the first 5 pages
Is there a pattern in the URL that we can exploit?

In [4]:
links = []
for page in range(1,6,1):
    url = f'https://www.jobindex.dk/jobsoegning?page={page}'
    links.append(url)

In [5]:
links

['https://www.jobindex.dk/jobsoegning?page=1',
 'https://www.jobindex.dk/jobsoegning?page=2',
 'https://www.jobindex.dk/jobsoegning?page=3',
 'https://www.jobindex.dk/jobsoegning?page=4',
 'https://www.jobindex.dk/jobsoegning?page=5']

## 2. Connect to the webpage and *download* its content

### Here is how you connect to and download the HTML of a webpage

In [6]:
import requests
response = requests.get('https://www.jobindex.dk/jobsoegning?page=1')

In [2]:
#response.text

#### Remember to tell who you are
- Write your name and email in the header of the request you send to the webpage
- Then the managers of the webpage will know you are not a malicious actor

In [3]:
response = requests.get('https://www.jobindex.dk/jobsoegning?page=1', headers={'name':'Hjalte Fejerskov Boas','email':'hfb@econ.ku.dk'})

NameError: name 'requests' is not defined

### We now want to download the content of all 5 links we made earlier

In [ ]:
list_htmls = []
for url in links:
    response = requests.get(url)
    html = response.text
    list_htmls.append(html)

In [4]:
#list_htmls

#### It is good practice to limit the rate of your calls to the website
You can do that with the function `time.sleep()`

In [5]:
list_htmls = []
for url in tqdm.tqdm(links): #Track the time left before completing the loop
    response = requests.get(url)
    html = response.text
    list_htmls.append(html)
    time.sleep(0.5) #Sleep for 0.5 seconds

NameError: name 'tqdm' is not defined

# Video 6.2: Logging and handling exceptions

## Logging
Logging your web scraping activity is crucial to ensuring and demonstrating data quality. It helps you:
- to document the data you extract
- to understand the reasons behind any unexpected stops of your web scraper

### Minimum essentials to log 
- *Time* of the scrape
- The *status code* of the request response
    - If succesful, the status code is normally *200*
    - A common error is *404*: "*Page not found*"
- The *length* of the output
    - To indicate whether there may be a mistake
    - In our case the output is usually an HTML-string
- The *path* to the output file
    - What did we actually scrape?

#### Here is a simple logging function that you can use:

In [ ]:
# Define the log function to gather the log information
def log(response,logfile,output_path=os.getcwd()):
    # Open or create the csv file
    if os.path.isfile(logfile): #If the log file exists, open it and allow for changes     
        log = open(logfile,'a')
    else: #If the log file does not exist, create it and make headers for the log variables
        log = open(logfile,'w')
        header = ['timestamp','status_code','length','output_file']
        log.write(';'.join(header) + "\n") #Make the headers and jump to new line
        
    # Gather log information
    status_code = response.status_code #Status code from the request result
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())) #Local time
    length = len(response.text) #Length of the HTML-string
    
    # Open the log file and append the gathered log information
    with open(logfile,'a') as log:
        log.write(f'{timestamp};{status_code};{length};{output_path}' + "\n") #Append the information and jump to new line

NameError: name 'os' is not defined

#### Apply to web scrape:

In [ ]:
list_htmls = []
logfile = 'log.csv'
for url in tqdm.tqdm(links):
    response = requests.get(url)
    html = response.text
    list_htmls.append(html)
    time.sleep(0.5)
    log(response,logfile)

100%|█████████████████████████████████████████████| 5/5 [00:05<00:00,  1.19s/it]


## Handling exceptions
When you web scrape you will encounter unexpected errors or crashes.
Some common errors may be:
- The URL does not exist
- The connection to the internet stops

How can we mitigate such problems?
- The `Try/Except` block in python can help us
    - If the computer encounter an error then it will execute the except code block instead

In [ ]:
list_htmls = []
for url in tqdm.tqdm(links):
    try:
        response = requests.get(url)
    except Exception as e:
        print(url) #Print url
        print(e) #Print error
        with open("list_htmls", "w") as l: #Save the list_htmls as a json file to retrieve at another time
            json.dump(list_htmls, l)
        continue #Continue to next iteration of the loop
    html = response.text
    list_htmls.append(html)
    time.sleep(0.5) #Sleep for 0.5 seconds

100%|█████████████████████████████████████████████| 5/5 [00:06<00:00,  1.23s/it]


# Video 6.3: The network panel

## Background
Many webpages are built dynamically 
- Each time you open up the webpage, it sends some **requests** to the server to retrieve the data that you see on the webpage

We can find these requests and use them!
- We can then send them directly to the server
- And then extract the data *before* it is written into the HTML
    - Much more preferable: The data usually comes in a structured JSON format
    - Makes it easy to use the data right away

## The network panel in Chrome Developer Tools
Use the **network panel** in the Chrome Developer Tools
- The network panel monitors all uploads and downloads to and from the webpage
- I.e. also requests about data we are interested in

### The network panel: [www.boligsiden.dk](https://www.boligsiden.dk/tilsalg)
<img src="https://drive.google.com/uc?export=view&id=1vGk2b1jxH1LU642-YWo1QgR3Q5vz2rQe">

### Which request is the one?
1. We want to locate an XHR ([XMLHttpRequest](https://en.wikipedia.org/wiki/XMLHttpRequest)). The XHRs transfer data between the server and the webpage 
    - Pick "*Fetch/XHR*"
2. Which XHR carries the information about the properties?
    - We need to look through them all --> You can get a preview of the JSON file in "*Preview*"
3. When we have found the right XHR, we need to find the request URL
    - Go to "*Headers*". Here you see the request URL

### The steps:

#### Send the request to the server and get the data

In [ ]:
response = requests.get('https://api.prod.bs-aws-stage.com/search/cases?addressTypes=villa%2Ccondo%2Cterraced+house%2Choliday+house%2Ccooperative%2Cfarm%2Chobby+farm%2Cfull+year+plot%2Cvilla+apartment%2Choliday+plot&per_page=50&page=1&highlighted=true&sortAscending=true&sortBy=timeOnMarket')

#### Convert the data to a JSON format in Python

In [ ]:
result = response.json()

#### The JSON file consists of two different key-value pairs ("cases" and "links"). 
#### We are only interested in the information about properties which are stored in the "cases" key-value pair

In [ ]:
result_properties = result['cases']

#### Now we can easily convert the JSON file to a pandas DataFrame

In [ ]:
data = pd.DataFrame(result_properties)

In [ ]:
data

,_links,address,addressType,caseID,caseUrl,coordinates,daysOnMarket,defaultImage,descriptionBody,descriptionTitle,...,status,timeOnMarket,totalClickCount,totalFavourites,utilitiesConnectionFee,weightedArea,yearBuilt,basementArea,secondaryAddressType,cooperative
0,{'self': {'href': '/cases/1b9cf386-67ef-4b81-9...,{'_links': {'self': {'href': '/addresses/0a3f5...,villa,1b9cf386-67ef-4b81-9d97-54d0172ea237,http://www.nybolig.dk/maegler/pages/property-p...,"{'lat': 55.250706, 'lon': 11.7673, 'type': 'EP...",1,"{'imageSources': [{'size': {'height': 80, 'wid...","Drømmer I om en skøn villa, hvor I får plads a...","Stor, mulighedsrig villa i attraktive Holsted ...",...,open,"{'current': {'days': 1}, 'total': {'days': 1, ...",197,2,{'includedInPrice': None},220.00,1967,NaN,NaN,NaN
1,{'self': {'href': '/cases/6b676471-9999-43c6-8...,{'_links': {'self': {'href': '/addresses/0a3f5...,villa,6b676471-9999-43c6-8361-e40b56c1b326,https://home.dk/sag/9030001331,"{'lat': 55.31548, 'lon': 10.795782, 'type': 'E...",1,"{'imageSources': [{'size': {'height': 80, 'wid...",Det her er sjældent at finde.... Adressen er B...,Nyborg,...,open,"{'current': {'days': 1}, 'total': {'days': 1, ...",406,2,{'includedInPrice': None},175.00,1924,83.0,NaN,NaN
2,{'self': {'href': '/cases/2faa5ba7-9a12-4007-8...,{'_links': {'self': {'href': '/addresses/0a3f5...,villa,2faa5ba7-9a12-4007-8738-efbc2a3ae512,https://www.danbolig.dk?propertyid=0140001772&...,"{'lat': 55.766117, 'lon': 11.673014, 'type': '...",1,"{'imageSources': [{'size': {'height': 80, 'wid...",I fantastiske omgivelser og med udsigt over vi...,Indflytningsklar villa på naturskønne Tuse Næs,...,open,"{'current': {'days': 1}, 'total': {'days': 1, ...",299,0,{'includedInPrice': None},111.95,1937,3.0,NaN,NaN
3,{'self': {'href': '/cases/7556a1d6-5e31-48bb-9...,{'_links': {'self': {'href': '/addresses/0a3f5...,villa,7556a1d6-5e31-48bb-961a-e88700a8a347,https://www.danbolig.dk?propertyid=0830000193&...,"{'lat': 55.942894, 'lon': 12.383393, 'type': '...",2,"{'imageSources': [{'size': {'height': 80, 'wid...","Grønne, smukke og landlige omgivelser skaber h...",Moderniseret og indflytningsklar villa i natur...,...,open,"{'current': {'days': 2}, 'total': {'days': 178...",570,7,{'includedInPrice': None},201.15,1977,78.0,NaN,NaN
4,{'self': {'href': '/cases/92aefbc3-4d54-43b3-9...,{'_links': {'self': {'href': '/addresses/0a3f5...,farm,92aefbc3-4d54-43b3-9530-df92d4fe44a4,https://www.danbolig.dk?propertyid=0140001812&...,"{'lat': 55.686302, 'lon': 11.713185, 'type': '...",2,"{'imageSources': [{'size': {'height': 80, 'wid...",Tæt på Holbæk har I nu mulighed for at overtag...,Indflytningsklar landejendom med mange mulighe...,...,open,"{'current': {'days': 2}, 'total': {'days': 167...",429,3,{'includedInPrice': None},313.45,1870,NaN,NaN,NaN
5,{'self': {'href': '/cases/8b9ae609-4d7e-4478-8...,{'_links': {'self': {'href': '/addresses/fa429...,villa,8b9ae609-4d7e-4478-8167-8eec15382f39,https://www.danbolig.dk?propertyid=0140001022&...,"{'lat': 55.54793, 'lon': 11.721237, 'type': 'E...",2,"{'imageSources': [{'size': {'height': 80, 'wid...",På en lukket vej med nem adgang til hverdagens...,Helt nyt hus med lave varmeudgifter og god ind...,...,open,"{'current': {'days': 2}, 'total': {'days': 2, ...",129,0,{'includedInPrice': None},172.85,2022,NaN,NaN,NaN
6,{'self': {'href': '/cases/c437b537-b585-4caf-9...,{'_links': {'self': {'href': '/addresses/0a3f5...,holiday house,c437b537-b585-4caf-94e5-b67c7f0bd9b5,https://www.danbolig.dk?propertyid=0140001805&...,"{'lat': 55.778934, 'lon': 11.805573, 'type': '...",2,"{'imageSources': [{'size': {'height': 80, 'wid...",I Salvig på Orø finder I dette renoverede somm...,"Flot, moderniseret sommerhus på Orø",...,open,"{'current': {'days': 2}, 'total': {'days': 2, ...",589,5,{'includedInPrice': None},38.80,1970,NaN,NaN,NaN
7,{'self': {'href': '/cases/b25571ce-7878-4f09-8...,{'_links': {'self': {'href': '/addresses/0a3f5...,condo,b25571ce-7878-4f09-817c-06baef565884,https://www.danbolig.dk?proper

## And we have the data!